# Step 2: Viterbi algorithm

In [105]:
import matplotlib.pyplot as plt
from random import randint

We begin by reloading the Data_Munging file

In [106]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


In [107]:
transitionDistribution

,0,1,4,5,10,13,15,17
0,0.990937,0.001203,0.226496,0.094737,0.001263,0.202703,0.026201,0.555556
1,0.001660,0.998231,0.004274,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.004151,0.000425,0.636752,0.000000,0.002685,0.000000,0.004367,0.000000
5,0.000899,0.000000,0.021368,0.905263,0.000000,0.000000,0.000000,0.000000
10,0.000553,0.000000,0.081197,0.000000,0.995736,0.000000,0.000000,0.000000
13,0.000553,0.000000,0.017094,0.000000,0.000316,0.797297,0.000000,0.037037
15,0.000277,0.000142,0.008547,0.000000,0.000000,0.000000,0.965066,0.000000
17,0.000969,0.000000,0.004274,0.000000,0.000000,0.000000,0.004367,0.407407


In [108]:
observationDistribution

,0,1,4,5,10,13,15,17
1,0.002905,0.000000,0.000000,0.000000,0.000000,0.067568,0.056769,0.000000
5,0.016947,0.000283,0.012821,0.878947,0.151453,0.000000,0.000000,0.000000
6,0.041779,0.000354,0.226496,0.000000,0.023215,0.000000,0.004367,0.000000
7,0.005257,0.000000,0.000000,0.000000,0.000000,0.000000,0.004367,0.111111
8,0.171751,0.000142,0.004274,0.000000,0.000000,0.297297,0.091703,0.777778
9,0.003113,0.000000,0.000000,0.000000,0.000000,0.040541,0.257642,0.000000
12,0.440963,0.999080,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
13,0.008854,0.000000,0.000000,0.000000,0.000000,0.000000,0.017467,0.000000
14,0.084388,0.000000,0.517094,0.121053,0.109128,0.081081,0.000000,0.000000
17,0.009615,0.000142,0.000000,0.000000,0.000000,0.162162,0.100437,0.000000


In [109]:
initialDistribution = Series([0,0,0.2,0.1,0.5,0.1,0,0.1], index = [0,1,4,5,10,13,15,17])
initialDistribution

0     0.0
1     0.0
4     0.2
5     0.1
10    0.5
13    0.1
15    0.0
17    0.1
dtype: float64

In [110]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [111]:
observation = Series([24,24,5,6,14,14,6,9,9,8,8,1,23,1,23,13,9,18,7,9,5,5,12,12,8,23,8,6,14,6,18,18,18,12,18,8,8,7,7,7,8,8,6,6,14,6])
observation

0     24
1     24
2      5
3      6
4     14
5     14
6      6
7      9
8      9
9      8
10     8
11     1
12    23
13     1
14    23
15    13
16     9
17    18
18     7
19     9
20     5
21     5
22    12
23    12
24     8
25    23
26     8
27     6
28    14
29     6
30    18
31    18
32    18
33    12
34    18
35     8
36     8
37     7
38     7
39     7
40     8
41     8
42     6
43     6
44    14
45     6
dtype: int64

In [112]:
def Viterbi(observation, activityLabel, initialDistribution, observationDistribution, transitionDistribution):
    probabilityTable = pd.DataFrame(0.0, index = activityLabel.index, columns = observation.index)
    path = {}
    for activityIndex in activityLabel.index.values:
        probabilityTable[0][activityIndex] = initialDistribution[activityIndex] * observationDistribution[activityIndex][observation[0]]
        path[activityIndex] = [activityIndex]
    
    for observationIndex in range(1,len(observation)):
        newPath = {}
        for activityIndex in activityLabel.index.values:
            (proba, previousActivityInd) = max((probabilityTable[observationIndex-1][previousActivityIndex]*transitionDistribution[previousActivityIndex][activityIndex]*observationDistribution[activityIndex][observation[observationIndex]], previousActivityIndex) for previousActivityIndex in activityLabel.index.values )
            #print proba 
            probabilityTable[observationIndex][activityIndex] = proba
            newPath[activityIndex] = path[previousActivityInd] + [activityIndex]
        path = newPath
    
    (finalProba, finalActivityIndex) = max((probabilityTable[len(observation)-1][activityIndex], activityIndex) for activityIndex in activityLabel.index.values )
    return finalProba, path[finalActivityIndex]
        
        
        

Now we will create the testing set:

In [113]:
daySensor = [sensorData['Start time'][x].day == 27 and sensorData['End time'][x].day == 27 for x in range(1,len(sensorData))]
daySensor = daySensor + [False]
daySensorData = sensorData[daySensor]
daySensorData.index = np.arange(1,len(daySensorData)+1)
daySensorData


,Start time,End time,ID,Val
1,2008-02-27 05:29:23,2008-02-27 05:30:24,24,1
2,2008-02-27 05:29:30,2008-02-27 05:29:33,6,1
3,2008-02-27 05:30:14,2008-02-27 05:30:15,14,1
4,2008-02-27 05:30:16,2008-02-27 05:30:22,6,1
5,2008-02-27 08:00:17,2008-02-27 08:00:18,24,1
6,2008-02-27 08:01:25,2008-02-27 08:01:29,9,1
7,2008-02-27 08:01:31,2008-02-27 08:01:50,23,1
8,2008-02-27 08:01:52,2008-02-27 08:01:53,8,1
9,2008-02-27 08:01:57,2008-02-27 08:02:08,8,1
10,2008-02-27 08:13:30,2008-02-27 08:13:31,6,1


In [114]:
dayActivity = [activityData['Start time'][x].day == 27  and activityData['End time'][x].day == 27 for x in range(1,len(activityData))]
dayActivity = dayActivity + [False]
dayActivityData = activityData[dayActivity]
dayActivityData.index = np.arange(1,len(dayActivityData)+1)
dayActivityData

,Start time,End time,ID
1,2008-02-27 05:29:11,2008-02-27 05:30:26,4
2,2008-02-27 08:01:22,2008-02-27 08:02:58,13
3,2008-02-27 08:13:55,2008-02-27 08:16:07,4
4,2008-02-27 08:16:45,2008-02-27 08:27:20,5
5,2008-02-27 08:34:25,2008-02-27 18:59:03,1
6,2008-02-27 18:59:55,2008-02-27 19:19:08,15
7,2008-02-27 19:19:52,2008-02-27 19:20:11,17
8,2008-02-27 23:03:23,2008-02-27 23:05:40,4


In [115]:
daySensorData['Start time'][1]

Timestamp('2008-02-27 05:29:23')

In [116]:
dayActivityData['Start time'][1]

Timestamp('2008-02-27 05:29:11')

In [117]:
testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)

In [118]:
testingFeatureMatrix

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1082.0,1083.0,1084.0,1085.0,1086.0,1087.0,1088.0,1089.0,1090.0,1091.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
t = testingFeatureMatrix.ix[:,:10]
t

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0


In [126]:
def crossValidation():
    dayList = testDay
    for d in dayList:
        print d
        daySensor = [sensorData['Start time'][x].day == d and sensorData['End time'][x].day == d for x in range(1,len(sensorData))]
        daySensor = daySensor + [False]
        daySensorData = sensorData[daySensor]
        daySensorData.index = np.arange(1,len(daySensorData)+1)
        
        dayActivity = [activityData['Start time'][x].day == d and activityData['End time'][x].day == d for x in range(1,len(activityData))]
        dayActivity = dayActivity + [False]
        dayActivityData = activityData[dayActivity]
        dayActivityData.index = np.arange(1,len(dayActivityData)+1)
        
        testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)
        
        testingObservation = Series(0, index = testingFeatureMatrix.columns)
        for i in range(len(testingObservation)): 
            r = np.where(testingFeatureMatrix[i] == 1)
            result = testingFeatureMatrix.index[r[0]]
            testingObservation[i] = result[0]
            
        errorRate = []
        numOfEvents = []
        for x in range(1,len(testingObservation),150): 
            obs = testingObservation[:x]
            validatedPath = testingLabel[:x]
            proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
            errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
            numOfEvents.append(x)
        plt.plot(numOfEvents,errorRate, label = d)
         
    plt.xlabel("Number of observations in the training sequence")
    plt.ylabel("Error rate (%)")
    plt.legend(loc='upper right')
    plt.show()

In [127]:
crossValidation()

25
28
5
10
16
